# Create Heat Map for significant Gene Expression genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,123/193,3.047452e-59,4.662602e-56,0,0,4.717289,635.599053,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,452/1631,6.421603e-57,4.912526e-54,0,0,2.051300,265.412919,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,92/134,8.907630e-49,4.542892e-46,0,0,5.081918,562.262229,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,92/144,1.076002e-44,4.115706e-42,0,0,4.729007,478.767028,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,124/348,5.502170e-26,1.683664e-23,0,0,2.637467,153.400560,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016


# Step 2: Get the list of significant genes 

In [4]:
trans = prot_enr.res2d.Genes[1]
genes = trans.split(';')
print('total genes:',len(genes))

total genes: 452


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

452

In [6]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=500, plot_width= 4500)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'USP39', 'RFC5', 'HNRNPD', 'ZKSCAN1', 'NUP37', 'PUS7',
       'YBX1', 'NUP153', 'RPS15A', 'EZH2', 'SSRP1', 'DDX39A', 'TEAD1',
       'PRMT1', 'CD2BP2', 'SESN1', 'ADAR', 'PARP1', 'RFC2', 'ZC3H8',
       'DEK', 'WDR75', 'BYSL', 'RFC4', 'RPN1', 'XPO5', 'TOPBP1', 'RPL39',
       'CHEK1', 'ZNF776', 'HEATR1', 'HNRNPA1', 'FANCI', 'TEX10', 'DIS3',
       'NFIA', 'NR2F1', 'RPL9', 'GTF2E2', 'ZNF655', 'RPS24', 'PPP2R2A',
       'NAT10', 'HDAC1', 'MSH2', 'DCAF13', 'DDOST', 'TPX2', 'AKT2',
       'RPS26', 'WDR46', 'GTF2F1', 'IMP4', 'RNMT', 'IGF2BP3', 'ATF2',
       'NFIC', 'NUAK1', 'IMP3', 'EXOSC9', 'E2F5', 'HNRNPU', 'RARA', 'ELL',
       'GSR', 'EMG1', 'ALYREF', 'TRMT6', 'DKC1', 'G6PD', 'MPHOSPH10',
       'DDX47', 'SP1', 'HNRNPA3', 'UTP6', 'NUP210', 'CDKAL1', 'TOP3A',
       'TFB2M', 'UTP3', 'RPL4', 'TSNAX', 'DDX52', 'TXNRD1', 'NOP56',
       'TDRKH', 'PSMB8', 'DARS2', 'FAS', 'GTPBP3', 'TP53', 'PSMB9',
       'IGF2BP2', 'ISG20L2', 'TAF1B', 'SUV39H1'], dtype=object)

In [9]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1500)

Loading BokehJS ...

# Sig in multiple cancers

In [10]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [11]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,101/1631,1.775164e-24,2.716001e-21,0,0,3.013392,164.796815,EHMT1;EPRS;PWP2;EFTUD2;PSMD9;SNRPD2;KHSRP;TXNL...,Reactome_2016
1,Processing of Capped Intron-Containing Pre-mRN...,36/193,3.337592e-24,2.553258e-21,0,0,9.076813,490.663374,DDX5;DHX9;DDX23;USP39;PRPF8;ELAVL1;NUP160;EFTU...,Reactome_2016
2,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",47/462,1.082883e-19,5.522702e-17,0,0,4.950443,216.183318,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
3,Cell Cycle Homo sapiens R-HSA-1640170,48/566,7.248901e-17,2.772705e-14,0,0,4.126796,153.364511,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
4,mRNA Splicing - Major Pathway Homo sapiens R-H...,24/134,4.208022e-16,1.287655e-13,0,0,8.715546,308.568418,DDX5;CPSF1;NCBP1;DHX9;NCBP2;CPSF3;CPSF2;DDX23;...,Reactome_2016


In [12]:
trans2 = enr2.res2d.Genes[0]
genes_mult = trans2.split(';')
print('total genes:',len(genes_mult))

total genes: 101


In [13]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
len(plot_df2.Proteomics.unique())

101

In [14]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1500)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [15]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['USP39', 'RFC5', 'PUS7', 'SSRP1', 'DDX39A', 'ADAR', 'PARP1',
       'RFC2', 'ZC3H8', 'WDR75', 'BYSL', 'RFC4', 'XPO5', 'TOPBP1',
       'HEATR1', 'NAT10', 'MSH2', 'DCAF13', 'TPX2', 'WDR46', 'IMP4',
       'RNMT', 'IMP3', 'EXOSC9', 'GSR', 'EMG1', 'MPHOSPH10', 'UTP6',
       'NUP210', 'TFB2M', 'DARS2', 'FAS'], dtype=object)

In [21]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=850)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [17]:

gnm = list(genes_pn_mult.Proteomics.unique())
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [18]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,32/1631,1.101718e-35,1.685628e-32,0,0,12.262416,987.046078,RNMT;HEATR1;NAT10;ZC3H8;ADAR;USP39;WDR46;EMG1;...,Reactome_2016
1,rRNA modification in the nucleus Homo sapiens ...,10/58,1.138526e-18,8.709721e-16,0,0,107.758621,4452.241117,UTP6;EMG1;IMP3;HEATR1;WDR75;NAT10;MPHOSPH10;IM...,Reactome_2016
2,rRNA processing Homo sapiens R-HSA-72312,12/180,3.761485e-17,1.918358e-14,0,0,41.666667,1575.797193,UTP6;EMG1;IMP3;EXOSC9;HEATR1;WDR75;NAT10;MPHOS...,Reactome_2016
3,Major pathway of rRNA processing in the nucleo...,11/166,1.021871e-15,3.908656e-13,0,0,41.415663,1429.550280,UTP6;EMG1;IMP3;EXOSC9;HEATR1;WDR75;MPHOSPH10;I...,Reactome_2016
4,Transcriptional Regulation by TP53 Homo sapien...,9/348,2.599148e-09,7.953393e-07,0,0,16.163793,319.527189,RFC5;TPX2;RFC4;MSH2;RFC2;GSR;FAS;SSRP1;TOPBP1,Reactome_2016


In [19]:
i = 4
print(enr3.res2d.Term[i])
enr3.res2d.Genes[i]

Transcriptional Regulation by TP53 Homo sapiens R-HSA-3700989


'RFC5;TPX2;RFC4;MSH2;RFC2;GSR;FAS;SSRP1;TOPBP1'